In [3]:
import numpy as np 
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

In [5]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, np_X_file_paths,np_y_file_paths):
        self.X_files = np_X_file_paths
        self.y_files = np_y_file_paths

    
    def __getitem__(self, index):
        x = np.load(self.X_files[index:index+self.tw])
        x = np.resize(x,(40,-1))
        x = torch.from_numpy(x).float()
        y = np.load(self.y_files[index+self.tw])
        y = torch.from_numpy(y).float()
        
        return x,y
    
    def __len__(self):
        return len(self.files)

    
'''class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)
    
'''
#train_dataset = MyDataset(x_train,y_train)
#vaild_dataset = 
    
from torch.utils.data import DataLoader 
#def collate_fn(batch):
    
    
#train_loader = DataLoader(train_dataset,shuffle=True,batch_size=16)

In [4]:
class LSTM(nn.Module):
    def __init__(self, input_size=40, hidden_layer_size=100, output_size=1,bidirectional=True):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size=input_size,
            hidden_size=hidden_layer_size,
            bidirectional=bidirectional)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return self.sigmoid(predictions[-1])